In [ ]:
!pip install transformers

     |████████████████████████████████| 2.2MB 5.6MB/s 
     |████████████████████████████████| 870kB 35.5MB/s 
     |████████████████████████████████| 3.3MB 39.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=4f7fb75b1ee078f62c4a015a63dbab569797afb6aab27c625d3f024f20f1a8bf
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
import pandas as pd
import math
from sklearn import preprocessing





task = 'task_1'

#2019 datasets 

df_test = pd.read_csv("/content/hasoc2019_hi_test_gold_2919.tsv",sep='\t')
df_train = pd.read_csv("/content/hindi_dataset.tsv",sep="\t")
df_train = df_train.dropna()



print(len(df_train))
print(df_train.head())

total_sentences = list(df_train['text'].values)
total_labels = list(df_train[task].values)



test_sentences = list(df_test['text'].values)
test_labels = list(df_test[task].values)

def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print(set(encoded_labels))

print(len(total_sentences),len(encoded_labels),len(test_sentences),len(encoded_test_labels))

print(df_test)

4665
         text_id  ... task_3
0  hasoc_hi_5556  ...   NONE
1  hasoc_hi_5648  ...    UNT
2   hasoc_hi_164  ...    TIN
3  hasoc_hi_3530  ...   NONE
4  hasoc_hi_5206  ...   NONE

[5 rows x 5 columns]
{0, 1}
4665 4665 1318 1318
            text_id  ... task_3
0     hasoc_hi_5061  ...   NONE
1     hasoc_hi_2090  ...    TIN
2     hasoc_hi_2960  ...    TIN
3      hasoc_hi_864  ...   NONE
4       hasoc_hi_54  ...   NONE
...             ...  ...    ...
1313  hasoc_hi_2773  ...    UNT
1314  hasoc_hi_7231  ...    TIN
1315   hasoc_hi_769  ...    TIN
1316  hasoc_hi_5725  ...    UNT
1317  hasoc_hi_2519  ...   NONE

[1318 rows x 5 columns]


In [ ]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.


max token length is:  297


In [ ]:
def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=128,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  बांग्लादेश की शानदार वापसी, भारत को 314 रन पर रोका #INDvBAN #CWC19
Token IDs: tensor([  101, 67260, 45470, 10826,   896, 21202, 55904, 37038, 92191, 10914,
          117, 14311, 11267, 32031,   891, 11453, 12213,   891, 69897, 11208,
          108, 36351, 11490, 10477, 35999, 11537,   108, 63216, 10858, 54055,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,    

In [ ]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 3498
valid data samples is 1167
test data samples is 1318


In [ ]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
#model.cpu()
device = "cuda:0"
model = model.to(device)
model.cuda()


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = AdamW(model.parameters(),lr=2e-5,eps=1e-8)

from transformers import get_linear_schedule_with_warmup

epochs=10
total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [ ]:
import numpy as np

def predictions_labels(preds,labels):
    #print(preds.device,labels.device)
    pred = torch.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
import random
import numpy as np
import time
from sklearn.metrics import classification_report,accuracy_score,f1_score

total_t0 = time.time()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    #print(correct.device)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)

def predictions_labels(preds,labels):
    #print(preds.device,labels.device)
    pred = torch.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label

In [ ]:
def train():
  total_train_loss = 0
  total_train_acc = 0
    
  model.train() # set model in train mode for batchnorm and dropout layers in bert model
    
  for step,batch in enumerate(train_data_loader):
    #print("**************************************************************************")
    #print("Step : ",step,"  batch",len(batch))
    #print("**************************************************************************")
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    model.zero_grad()
    #loss,logits = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels.long())
    outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels.long())
    loss = outputs.loss
    logits = outputs.logits
    #total_train_loss+=loss.detach().numpy()
    total_train_loss+=loss.detach()
    total_train_acc+=categorical_accuracy(logits,b_labels).item()
            
    loss.backward()
            
    torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            
    optimizer.step()
            
    scheduler.step() #go ahead and update the learning rate
    #print(total_train_loss,total_train_acc)
            
  avg_train_loss = total_train_loss/len(train_data_loader)
  avg_train_acc = total_train_acc/len(train_data_loader)
    
  return avg_train_loss,avg_train_acc

In [ ]:
def evaluate():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []

    for batch in valid_data_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

        #loss, logits = model(b_input_ids,attention_mask= b_input_mask,labels = b_labels.long())
          outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels.long())
        loss = outputs.loss
        logits = outputs.logits

        #total_eval_loss+=loss.detach().numpy()

        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()

        total_eval_loss+=loss.detach()        
        logits = logits.detach()
        label_ids = b_labels.to(device)

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred.detach().cpu().numpy())
        all_true_labels.extend(true.detach().cpu().numpy())
    
    #print(np.shape(np.array(all_pred_labels).reshape(-1,1)),np.shape(np.array(all_true_labels).reshape(-1,1)))

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,macro_f1_score

In [ ]:
import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
epochs = 10
train_loss = 0
train_acc = 0
valid_loss = 0
valid_acc = 0
macro_f1  = 0
best_macro_f1 = float('0')
for epoch in range(epochs):
  start_time = time.time()
  train_loss,train_acc = train()
  valid_loss,valid_acc,macro_f1 = evaluate()
    
  end_time = time.time()
        
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
  if macro_f1 > best_macro_f1:
    best_macro_f1 = macro_f1
    torch.save(model,'model_english_task_a.pt')
  
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
  print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
  #print(f'\t macro_f1: {macro_f1:.3f} |  c: {valid_acc*100:.2f}%')

              precision    recall  f1-score   support

           0       0.81      0.84      0.83       604
           1       0.82      0.79      0.81       563

    accuracy                           0.82      1167
   macro avg       0.82      0.82      0.82      1167
weighted avg       0.82      0.82      0.82      1167

accuracy = 0.82
Epoch: 01 | Epoch Time: 3m 36s
	Train Loss: 0.505 | Train Acc: 75.57%
	 Val. Loss: 0.441 |  Val. Acc: 81.75%
              precision    recall  f1-score   support

           0       0.89      0.75      0.82       741
           1       0.66      0.85      0.74       426

    accuracy                           0.79      1167
   macro avg       0.78      0.80      0.78      1167
weighted avg       0.81      0.79      0.79      1167

accuracy = 0.79
Epoch: 02 | Epoch Time: 3m 36s
	Train Loss: 0.401 | Train Acc: 83.65%
	 Val. Loss: 0.712 |  Val. Acc: 78.58%
              precision    recall  f1-score   support

           0       0.86      0.81      0.

In [ ]:
del model
import gc
gc.collect()
  
model = torch.load('model_hindi_task_a.pt')
model = model.to(device)

In [ ]:
def evaluate_test():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []
    
    all_sentence_id=[]

    for batch in test_data_loader:
        b_sentence_id = batch[0].to(device)
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        sent_ids = b_sentence_id.to('cpu').numpy()
        all_sentence_id.extend(sent_ids)
        
        with torch.no_grad():

            outputs = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long())
        
        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss+=loss.item()

        logits = logits.detach().cpu()

        label_ids = b_labels.to('cpu')
        

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,all_sentence_id,all_pred_labels

valid_loss,valid_acc,all_sentence_id,all_pred_labels = evaluate_test()


              precision    recall  f1-score   support

           0       0.82      0.55      0.66       424
           1       0.78      0.93      0.85       729

    accuracy                           0.79      1153
   macro avg       0.80      0.74      0.75      1153
weighted avg       0.79      0.79      0.78      1153

accuracy = 0.79
